In [1]:
import time
import cv2
import numpy as np
from tsdf import TSDF, get_volume_bounds

PyCUDA successfully imported. Running in GPU mode.


In [17]:
data_folder = "data/chess-7scenes/"
cam_intrinsics_file = data_folder + "camera-intrinsics.txt"
frame_color_img_file = data_folder + "frame-{:06d}.color.png"
frame_depth_img_file = data_folder + "frame-{:06d}.depth.png"
frame_pose_file = data_folder + "frame-{:06d}.pose.txt"
frames_count = 1000

In [18]:
def process_data():
    frame_color_imgs = []
    frame_depth_imgs = []
    frame_poses = []

    # read the data
    for i in range(frames_count):
        color_img = cv2.imread(frame_color_img_file.format(i))
        color_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB)
        frame_color_imgs.append(color_img)
        depth_img = (
            cv2.imread(frame_depth_img_file.format(i), cv2.IMREAD_UNCHANGED) / 1000
        )
        depth_img[depth_img == 65.535] = 0
        frame_depth_imgs.append(depth_img)
        frame_poses.append(np.loadtxt(frame_pose_file.format(i)))
    return frame_color_imgs, frame_depth_imgs, frame_poses


In [20]:
frame_color_imgs, frame_depth_imgs, frame_poses = process_data()
cam_intrinsics = np.loadtxt(cam_intrinsics_file)

In [21]:
# tsdf
tsdf_voxel_size = 0.02  # 2cm
tsdf_trunc_margin = 0.1  # 10cm

volume_bounds = get_volume_bounds(
    frames_count, frame_depth_imgs, frame_poses, cam_intrinsics
)

Volume bounds:
[[-3.52754889  4.4481571 ]
 [-2.78254933  3.25919307]
 [-0.99043971  5.57913861]]


In [22]:
tsdf_obj = TSDF(
    volume_bounds,
    voxel_size=tsdf_voxel_size,
    trunc_margin=tsdf_trunc_margin,
    run_on_gpu=True,
)
start_time = time.time()
tsdf_obj.build_tsdf(
    frames_count, frame_color_imgs, frame_depth_imgs, frame_poses, cam_intrinsics
)
end_time = time.time()
print("Time: ", end_time - start_time, "seconds")
print("FPS: ", frames_count / (end_time - start_time))

Volume dims:  [399 303 329]
Using GPU:  NVIDIA GeForce RTX 3090
Maximum number of threads per block: 1024
Maximum block dimension: 1024, 1024, 64
Maximum grid dimension: 2147483647, 65535, 65535
Block size: (1024, 1, 1), Grid size: (38843, 1, 1)
Fusing frame 0
Fusing frame 1
Fusing frame 2
Fusing frame 3
Fusing frame 4
Fusing frame 5
Fusing frame 6
Fusing frame 7
Fusing frame 8
Fusing frame 9
Fusing frame 10
Fusing frame 11
Fusing frame 12
Fusing frame 13
Fusing frame 14
Fusing frame 15
Fusing frame 16
Fusing frame 17
Fusing frame 18
Fusing frame 19
Fusing frame 20
Fusing frame 21
Fusing frame 22
Fusing frame 23
Fusing frame 24
Fusing frame 25
Fusing frame 26
Fusing frame 27
Fusing frame 28
Fusing frame 29
Fusing frame 30
Fusing frame 31
Fusing frame 32
Fusing frame 33
Fusing frame 34
Fusing frame 35
Fusing frame 36
Fusing frame 37
Fusing frame 38
Fusing frame 39
Fusing frame 40
Fusing frame 41
Fusing frame 42
Fusing frame 43
Fusing frame 44
Fusing frame 45
Fusing frame 46
Fusing frame

In [23]:
tsdf_obj.save_mesh("outputs/chess_mesh.ply")
tsdf_obj.save_point_cloud("outputs/chess_pc.ply")

Mesh saved to outputs/chess_mesh.ply
Point cloud saved to outputs/chess_pc.ply
